In [309]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [310]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [311]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [312]:
%%capture pwd
!pwd

In [313]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('what-is-supply-chain-inflation-and-why-is-it-driving-up-consumer-prices-now',
 300,
 500)

# Fig 1

In [314]:
df = pd.read_csv("raw/CPIdecomp.csv")
df = df.dropna(subset=["goods_rate_w"])
df["date"] = (
    pd.to_datetime(df["date"], dayfirst=True)
    .astype(str)
    .str.replace("-11-", "-31-")
    .str.replace("-11", "-01")
    .str.replace("-31", "-11")
)
df = pd.concat(
    [
        pd.DataFrame(
            {
                "date": "2019-12-01",
                "all_rate": np.nan,
                "serv_rate_w": 0,
                "goods_rate_w": 0,
            },
            index=[99],
        ),
        df,
    ]
)

In [315]:
f = "fig1_cpi"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,all_rate,serv_rate_w,goods_rate_w
99,2019-12-01,NaN,0.000,0.000
0,2020-01-01,1.8,1.173,0.637
1,2020-02-01,1.7,1.275,0.490
2,2020-03-01,1.5,1.275,0.294
3,2020-04-01,0.8,1.020,-0.196


In [316]:
base = (
    alt.Chart(f1)
    .encode(
        x=alt.X(
            "date:T",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                tickCount=10,
                orient="bottom",
                labelAngle=0,
            ),
        )
    )
    .transform_calculate(start="max(0,datum.goods_rate_w)")
    .transform_calculate(end="datum.start+datum.serv_rate_w")
)
bars1 = base.mark_bar(color=colors["eco-turquiose"], width=15, opacity=0.8).encode(
    y=alt.Y(
        "start:Q",
        sort=[],
        axis=alt.Axis(
            grid=True,
            gridColor=colors["eco-gray"],
            gridOpacity=0.1,
            title="YoY % change",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            #                 offset=25,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=20,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".0f",
        ),
    ),
    y2="end:Q",
)
bars2 = base.mark_bar(color=colors["eco-red"], width=15, opacity=0.8).encode(
    y=alt.Y(
        "goods_rate_w:Q",
    )
)
bars2a = bars2.transform_filter("datum.goods_rate_w<0")
bars2b = bars2.transform_filter("datum.goods_rate_w>0")
line = base.mark_line(color=colors["eco-mid-blue"]).encode(y="all_rate:Q")

axis1 = (
    alt.Chart(pd.DataFrame([{"x": "2019-12-01", "y": 0}, {"x": "2021-11-01", "y": 0}]))
    .mark_line(strokeWidth=1, color=colors["eco-gray"], opacity=0.3)
    .encode(x=alt.X("x:T", sort=[]), y="y:Q")
)
axis2 = (
    alt.Chart(
        pd.DataFrame([{"x": "2021-01-01", "y": 0.95}, {"x": "2021-01-01", "y": 5.5}])
    )
    .mark_line(strokeWidth=1, color=colors["eco-gray"], opacity=0.3)
    .encode(x=alt.X("x:T", sort=[]), y="y:Q")
)
label1 = (
    alt.Chart(pd.DataFrame({"x": "2020-01-24", "y": 2, "t": "Services"}, index=[0]))
    .mark_text(color=colors["eco-turquiose"], size=12)
    .encode(x="x:T", y="y:Q", text="t:N")
)
label2 = (
    alt.Chart(pd.DataFrame({"x": "2020-01-17", "y": -0.2, "t": "Goods"}, index=[0]))
    .mark_text(color=colors["eco-red"], size=12)
    .encode(x="x:T", y="y:Q", text="t:N")
)
label3 = (
    alt.Chart(pd.DataFrame({"x": "2021-09-17", "y": 4.7, "t": "CPI"}, index=[0]))
    .mark_text(color=colors["eco-mid-blue"], size=12)
    .encode(x="x:T", y="y:Q", text="t:N")
)
layer1 = (
    (
        (
            bars1 + bars2a + bars2b + line + axis1 + axis2 + label1 + label2 + label3
        ).properties(height=300, width=450)
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [317]:
df = pd.read_excel("raw/oil&gas.xlsx", skiprows=2, header=None)
df.columns = ["gd", "g", "d", "od", "o"]
df1 = df[["gd", "g"]]
df1.columns = ["date", "gas"]
df2 = df[["od", "o"]]
df2.columns = ["date", "oil"]
df = df1.set_index("date").join(df2.set_index("date")).reset_index()

In [318]:
f = "fig2_oil_gas"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,gas,oil
0,2009-01-04,56.80,32.04
1,2009-01-05,60.75,33.37
2,2009-01-06,69.00,32.53
3,2009-01-07,66.50,28.07
4,2009-01-08,69.00,27.43


In [319]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line1 = base.mark_line(color=colors["eco-red"]).encode(
    y=alt.Y(
        "gas:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="pence per therm",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-red"],
            titleColor=colors["eco-red"],
            tickColor=colors["eco-red"],
            domainColor=colors["eco-red"],
        ),
    )
)
line2 = base.mark_line(color=colors["eco-blue"]).encode(
    y=alt.Y(
        "oil:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="£ per barrel",
            titleX=5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="right",
            labelColor=colors["eco-blue"],
            titleColor=colors["eco-blue"],
            tickColor=colors["eco-blue"],
            domainColor=colors["eco-blue"],
        ),
    )
).transform_filter('datum.oil!=0')
label1 = (
    alt.Chart(pd.DataFrame({"x": "2009-10-22", "y": -10, "t": "Gas"}, index=[0]))
    .mark_text(color=colors["eco-red"], size=12)
    .encode(x="x:T", y=alt.Y("y:Q"), text="t:N")
)
label2 = (
    alt.Chart(pd.DataFrame({"x": "2009-10-22", "y": 30, "t": "Oil"}, index=[0]))
    .mark_text(color=colors["eco-blue"], size=12)
    .encode(x="x:T", y=alt.Y("y:Q"), text="t:N")
)
layer1 = (
    (((line1) + (line2 + label2 + label1)).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
).resolve_scale(y="independent")
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 3

In [320]:
df = (
    pd.read_excel("raw/manufacturing.xls", skiprows=10, header=None)
    .dropna(axis=1, how="all")
    .dropna(subset=[3])
)
df.columns = ["date", "out", "in"]
df['out']=df['out'].astype(float)
df['in']=df['in'].astype(float)
df['date']=df['date'].astype(str)

In [321]:
f = "fig3_input_output"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,date,out,in
0,2015-01-01 00:00:00,-3.7,-6.4
1,2015-02-01 00:00:00,-3.5,-6.3
2,2015-03-01 00:00:00,-3.3,-6.2
3,2015-04-01 00:00:00,-3.2,-5.7
4,2015-05-01 00:00:00,-2.8,-6.0


In [322]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line1 = base.mark_line(color=colors["eco-blue"]).encode(
    y=alt.Y(
        "in:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    )
)
line2 = base.mark_line(color=colors["eco-turquiose"]).encode(y=alt.Y("out:Q", sort=[]))
label1 = (
    alt.Chart(pd.DataFrame({"x": "2017-11-22", "y": 9.5, "t": "Input prices"}, index=[0]))
    .mark_text(color=colors["eco-blue"], size=12)
    .encode(x="x:T", y=alt.Y("y:Q"), text="t:N")
)
label2 = (
    alt.Chart(pd.DataFrame({"x": "2017-04-22", "y": 0, "t": "Output prices"}, index=[0]))
    .mark_text(color=colors["eco-turquiose"], size=12)
    .encode(x="x:T", y=alt.Y("y:Q"), text="t:N")
)
layer1 = (
    ((line1+line2+label1+label2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)

layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)